<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f23c1c8e8db6435f72b1377da6281e183c3187da531145477c22caa92f2ca588
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-27 15:42:30
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -25.35 K (-0.18%)
Current PnL: -22.37 L (-14.66%)
CY Booked + Current PnL: -8.48 L (-5.56%)
-------------------
Total profit:  1.74 L
Total loss:  -24.11 L
-------------------
Total Booked + Current PnL: 18.50 L (14.73%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.68%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.78 L (61.2%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.52%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,0.00,-14.12,16.54,0.09,14316.0,-14225.0,86552.0,100.85,50.0,M-SC,3.54,253.0,-0.99,0.60,13.71,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.0,-1.97,-9.01,9.93,0.02,15771.0,-15734.0,158825.0,142.19,67.0,M-SC,9.15,234.0,-1.00,1.11,38.15,OX40N,NTT,PAINTS
83,VOLTAS,1530.0,0.79,9.37,9.44,19.69,19796.0,17958.0,209700.0,-1.35,54.0,X-MC,3.37,78.0,0.91,1.46,16.80,XY25,NTT,AC
43,ITC,452.0,0.50,-1.22,11.80,10.44,23329.0,-2435.0,197703.0,-41.15,46.0,X-LC,1.00,5.0,-0.10,1.38,4.28,X40,NTT,FMCG
52,MEDANTA,1486.0,-1.05,4.37,18.88,24.08,25016.0,5550.0,132500.0,-4.04,48.0,X-SC,5.56,89.0,0.22,0.92,24.85,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GILLETTE,11206.78,3.97,3.54,27.89,32.42,73320.0,8994.0,262890.0,-14.94,52.0,X-SC,7.53,85.0,0.12,1.83,19.18,X40,ATH,FMCG
58,RAJOOENG,143.10,3.51,-24.82,85.70,39.61,66040.0,-25440.0,77060.0,-49.12,40.0,H-SC,20.48,135.0,-0.39,0.54,8.61,AR,ATH,MISC
67,SIS,528.00,2.65,-20.82,53.15,21.26,46725.0,-23121.0,87911.0,2078.24,61.0,H-SC,3.78,166.0,-0.49,0.61,18.78,OX40N,NTT,MISC
50,LTIM,7201.88,2.30,10.15,19.52,31.66,52928.0,24997.0,271148.0,6.01,64.0,H-LC,13.21,43.0,0.47,1.89,50.45,X200,ATH,IT
21,DEN,75.00,1.78,-39.33,134.16,42.07,68754.0,-33216.0,51248.0,81.35,47.0,M-SC,8.16,235.0,-0.48,0.36,8.83,AR,NTT,ENTERTAINMENT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-11.42,-14.75,113.39,81.92,110975.0,-16928.0,97870.0,-50.12,23.0,M-SC,3.32,236.0,-0.15,0.68,15.86,XR,NTT,DURABLES
44,JCHAC,2282.0,-3.67,-27.57,38.04,-0.01,35215.0,-35231.0,92574.0,18834.78,16.0,M-SC,1.42,233.0,-1.00,0.65,4.83,OX40N,NTT,AC
28,GREENPANEL,537.0,-2.13,-32.63,112.46,43.14,138710.0,-59736.0,123342.0,176.49,32.0,M-SC,5.86,240.0,-0.43,0.86,16.02,XY24,NTT,MISC
70,SURYODAY,216.0,-1.99,-19.89,53.58,23.03,76862.0,-35618.0,143453.0,55.70,48.0,H-SC,9.45,167.0,-0.46,1.00,42.33,XR,NTT,BANKS
39,INDIGOPNTS,1408.0,-1.97,-9.01,9.93,0.02,15771.0,-15734.0,158825.0,142.19,67.0,M-SC,9.15,234.0,-1.00,1.11,38.15,OX40N,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.7,-0.17,107.08,106.72,131841.0,-212.0,123124.0,-50.63,37.0,H-SC,3.1,139.0,-0.0,0.86,23.17,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.36,-4.55,117.81,107.91,166033.0,-6711.0,140933.0,-22.17,37.0,M-SC,10.57,216.0,-0.04,0.98,0.78,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.97,-9.01,9.93,0.02,15771.0,-15734.0,158825.0,142.19,67.0,M-SC,9.15,234.0,-1.00,1.11,38.15,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.00,-14.12,16.54,0.09,14316.0,-14225.0,86552.0,100.85,50.0,M-SC,3.54,253.0,-0.99,0.60,13.71,OX40N,NTT,DURABLES
67,SIS,528.00,2.65,-20.82,53.15,21.26,46725.0,-23121.0,87911.0,2078.24,61.0,H-SC,3.78,166.0,-0.49,0.61,18.78,OX40N,NTT,MISC
47,KANSAINER,340.00,1.16,-20.99,43.62,13.47,92935.0,-56610.0,213057.0,-67.53,39.0,H-SC,1.47,158.0,-0.61,1.49,8.33,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.25,2.36,68.3,72.27,118462.0,3996.0,173444.0,-11.98,59.0,M-LC,3.87,99.0,0.03,1.21,8.68,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.25,2.36,68.30,72.27,118462.0,3996.0,173444.0,-11.98,59.0,M-LC,3.87,99.0,0.03,1.21,8.68,XR,NTT,IT
38,INDIAMART,4810.62,-0.70,-0.17,107.08,106.72,131841.0,-212.0,123124.0,-50.63,37.0,H-SC,3.10,139.0,-0.00,0.86,23.17,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,-0.35,-1.17,37.23,35.63,77115.0,-2442.0,207132.0,-14.97,41.0,H-MC,3.97,119.0,-0.03,1.44,15.31,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.36,-4.55,117.81,107.91,166033.0,-6711.0,140933.0,-22.17,37.0,M-SC,10.57,216.0,-0.04,0.98,0.78,OX40N,ATH,CABLES
37,IEX,219.00,-0.61,-4.38,55.43,48.63,105983.0,-8753.0,191201.0,-36.48,52.0,H-SC,15.62,136.0,-0.08,1.33,6.67,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.02,-35.49,109.58,35.20,90960.0,-45662.0,83008.0,8.88,24.0,X-SC,13.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,0.04,-9.08,26.74,15.23,48707.0,-18198.0,182149.0,-22.32,28.0,X-MC,6.92,66.0,-0.37,1.27,20.24,X40N,NTT,REALTY
76,TMPV,600.00,-0.40,-15.21,67.69,42.18,158638.0,-42051.0,234359.0,-24.30,28.0,X-LC,1.49,3.0,-0.27,1.63,1.52,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.33,-13.83,72.28,48.45,124881.0,-27732.0,172774.0,-20.09,32.0,X-MC,7.32,64.0,-0.22,1.20,0.37,X40N,ATH,FINANCE
55,PGHH,17907.65,0.03,-4.54,40.12,33.76,76910.0,-9120.0,191700.0,-32.64,33.0,X-MC,3.73,57.0,-0.12,1.34,0.42,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.50,-1.22,11.80,10.44,23329.0,-2435.0,197703.0,-41.15,46.0,X-LC,1.00,5.0,-0.10,1.38,4.28,X40,NTT,FMCG
3,ACC,3906.0,-0.76,-21.40,108.85,64.15,203571.0,-50931.0,187020.0,-54.13,53.0,X-SC,1.49,82.0,-0.25,1.30,4.91,XY24,BTT,CEMENT
76,TMPV,600.0,-0.40,-15.21,67.69,42.18,158638.0,-42051.0,234359.0,-24.30,28.0,X-LC,1.49,3.0,-0.27,1.63,1.52,XY24,NTT,AUTO
36,ICICIPRULI,790.0,0.54,3.66,26.35,30.97,48273.0,6460.0,183198.0,-20.69,59.0,X-MC,1.83,75.0,0.13,1.28,16.68,X40,ATH,INSURANCE
66,SIEMENS,4671.5,-0.20,-11.01,41.04,25.51,67964.0,-20490.0,165605.0,1.56,66.0,H-LC,1.86,49.0,-0.30,1.15,22.04,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.02,-35.49,109.58,35.20,90960.0,-45662.0,83008.0,8.88,24.0,X-SC,13.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,0.33,-13.83,72.28,48.45,124881.0,-27732.0,172774.0,-20.09,32.0,X-MC,7.32,64.0,-0.22,1.20,0.37,X40N,ATH,FINANCE
55,PGHH,17907.65,0.03,-4.54,40.12,33.76,76910.0,-9120.0,191700.0,-32.64,33.0,X-MC,3.73,57.0,-0.12,1.34,0.42,X40,ATH,FMCG
54,PAGEIND,51605.08,-0.18,-4.90,32.56,26.07,50702.0,-8020.0,155720.0,-29.94,39.0,X-MC,9.37,55.0,-0.16,1.09,1.03,X40,ATH,APPARELS
76,TMPV,600.00,-0.40,-15.21,67.69,42.18,158638.0,-42051.0,234359.0,-24.30,28.0,X-LC,1.49,3.0,-0.27,1.63,1.52,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.56,-28.01,98.43,42.84,46061.0,-18210.0,46796.0,-52.62,41.0,X-SC,37.57,83.0,-0.40,0.33,1.83,XY24,NTT,MISC
60,RELAXO,1176.00,-0.55,-46.88,186.72,52.31,143976.0,-68052.0,77108.0,-43.47,38.0,X-SC,4.43,91.0,-0.47,0.54,3.21,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.02,-35.49,109.58,35.20,90960.0,-45662.0,83008.0,8.88,24.0,X-SC,13.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.23,-5.96,29.29,21.58,38600.0,-8352.0,131787.0,-52.56,36.0,X-MC,6.58,56.0,-0.22,0.92,16.67,X40N,ATH,IT
52,MEDANTA,1486.00,-1.05,4.37,18.88,24.08,25016.0,5550.0,132500.0,-4.04,48.0,X-SC,5.56,89.0,0.22,0.92,24.85,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.83,-14.78,39.96,19.28,117818.0,-51116.0,294840.0,-25.57,58.0,X-LC,7.24,1.0,-0.43,2.06,9.00,X40,ATH,IT
41,INFY,2275.00,0.55,6.63,45.24,54.87,150940.0,20755.0,333643.0,-17.09,62.0,X-LC,2.62,2.0,0.14,2.33,15.47,X40,BTT,IT
76,TMPV,600.00,-0.40,-15.21,67.69,42.18,158638.0,-42051.0,234359.0,-24.30,28.0,X-LC,1.49,3.0,-0.27,1.63,1.52,XY24,NTT,AUTO
82,VBL,671.64,0.39,-5.61,43.73,35.67,130375.0,-17705.0,298137.0,-16.86,55.0,X-LC,5.76,4.0,-0.14,2.08,7.47,X40N,ATH,FMCG
43,ITC,452.00,0.50,-1.22,11.80,10.44,23329.0,-2435.0,197703.0,-41.15,46.0,X-LC,1.00,5.0,-0.10,1.38,4.28,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.79,-36.70,109.92,32.89,53723.0,-28332.0,48875.0,-703.29,65.0,L-MC,5.65,259.0,-0.53,0.34,34.60,XR,NTT,BANKS
6,ASIANTILES,137.00,0.71,-12.03,106.51,81.67,87900.0,-11283.0,82527.0,7422.22,54.0,L-SC,19.36,271.0,-0.13,0.58,59.82,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.45,-11.86,58.32,39.54,51999.0,-11997.0,89162.0,-31.03,67.0,M-SC,6.60,220.0,-0.23,0.62,25.73,AR,ATH,AUTO
70,SURYODAY,216.00,-1.99,-19.89,53.58,23.03,76862.0,-35618.0,143453.0,55.70,48.0,H-SC,9.45,167.0,-0.46,1.00,42.33,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-1.97,-9.01,9.93,0.02,15771.0,-15734.0,158825.0,142.19,67.0,M-SC,9.15,234.0,-1.00,1.11,38.15,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-1.97,-9.01,9.93,0.02,15771.0,-15734.0,158825.0,142.19,67.0,M-SC,9.15,234.0,-1.00,1.11,38.15,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.26,0.18,0.63,20.19,20.95,51154.0,1593.0,253361.0,-0.35,76.0,X-LC,14.54,31.0,0.03,1.77,36.49,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.79,-36.70,109.92,32.89,53723.0,-28332.0,48875.0,-703.29,65.0,L-MC,5.65,259.0,-0.53,0.34,34.60,XR,NTT,BANKS
0,5PAISA,593.00,-0.89,-35.11,74.00,12.90,104912.0,-76723.0,141773.0,109.13,57.0,H-SC,13.41,161.0,-0.73,0.99,17.88,OX40N,NTT,FINANCE
79,UNITDSPR,1644.00,-0.94,6.21,13.71,20.77,34094.0,14534.0,248678.0,-3.08,61.0,X-MC,3.76,62.0,0.43,1.73,13.55,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.85
1,25,44.75
2,50,75.59


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.52
MC    29.21
LC    25.28
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.81
X40      21.26
X40N     11.92
XR        9.53
XY25      9.12
AR        9.08
OX40N     7.65
X200      1.89
SR        0.98
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.91
X-MC    22.73
X-LC    20.05
M-SC    11.92
X-SC     8.28
H-MC     4.75
H-LC     3.04
L-SC     1.41
M-MC     1.39
M-LC     1.21
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.82,-4.40,38.16
IT,13.79,-14.33,73.35
FINANCE,9.46,-14.29,62.58
MISC,7.41,-22.18,72.23
ELECTRICAL,5.99,-9.08,49.03
PAINTS,5.98,-7.81,23.83
INSURANCE,4.42,0.37,34.75
PHARMA,3.88,-0.50,32.67
AUTO,3.36,-19.09,70.15


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3047543.0,21
XR,1296761.0,13
AR,1267606.0,10
X40,992483.0,14
X40N,738526.0,9
OX40N,694315.0,10
XY25,333808.0,6
SR,280244.0,2
MH,73531.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3548821.0,25
M-SC,1350439.0,15
X-MC,1223052.0,16
X-LC,889641.0,11
X-SC,731574.0,8
H-MC,395869.0,3
L-SC,259706.0,3
H-LC,120892.0,2
M-LC,118462.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1212666.0      6
           AR         872480.0      5
           XR         792867.0      7
M-SC       XY24       780401.0      6
X-MC       X40        438952.0      7
X-LC       X40        389251.0      5
X-MC       XY24       362470.0      3
H-SC       OX40N      317033.0      4
X-SC       X40N       292646.0      3
M-SC       OX40N      289357.0      5
H-SC       SR         280244.0      2
X-SC       XY24       274648.0      3
X-MC       X40N       246282.0      4
X-LC       XY24       227898.0      2
H-MC       AR         206409.0      2
X-LC       X40N       199598.0      2
H-MC       XY24       189460.0      1
X-MC       XY25       175348.0      2
L-SC       XR         171781.0      2
X-SC       X40        164280.0      2
M-SC       XR         159928.0      2
           AR         120753.0      2
M-LC       XR         118462.0      1
L-SC       OX40N       87925.0      1
H-SC       MH          73531.0      1
X-LC       XY25        72894.0      2
H-LC       AR          67964.0      1
L-MC       XR          53723.0      1
H-LC       X200        52928.0      1
M-MC       XY25        46064.0      1
L-LC       XY25        39502.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
